In [0]:
import datetime

import numpy as np
import pandas as pd
import os

import io

In [22]:
from google.colab import files

uploaded = files.upload()
uploaded = files.upload()

Saving group_type.xlsx to group_type.xlsx


# target

In [0]:
WORKF = pd.read_excel('Data-Thailand-HR_masked.xlsx', 'TH workforce', encoding='utf-8')

In [0]:
LEAVE_ACTIONS = [
    'LEAVE THE COMPANY',
    'Withdrawn Chg. (Migration)',
#     'RETIREMENT NOT PAID',
#     'INTERNAL MOVE',
#     'RETURN TO HOME COMPANY',
#     'CHANGE EMPLOYEE SITUATION',
#     'Hiring Active (Migration)',
#     'RE-ENTRY IN THE COMPANY',
#     'ENTRY IN THE COMPANY',
]

In [0]:
WORKF['Leave'] = WORKF['Personnel Action Text'].map({a:1 for a in LEAVE_ACTIONS}).fillna(0)

In [0]:
target = WORKF[['Serial Number', 'Leave']].set_index('Serial Number')

In [15]:
target.head()

,Leave
Serial Number,
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0


In [20]:
target.groupby(['Leave']).size().reset_index(name='counts')

,Leave,counts
0,0.0,2218
1,1.0,2820


# Feature Extraction (Sheet: Hire Type & Sources)

In [0]:
features = dict()

In [0]:
GTYPE_FILE = pd.ExcelFile('group_type.xlsx')

GTYPE = dict()
GTYPE['HireTypeGroup'] = pd.read_excel(GTYPE_FILE, 'HireType')

In [0]:
HIRE = pd.read_excel('Data-Thailand-HR_masked.xlsx', 'Hire Type & Sources').iloc[:, :5]

HIRE.set_index(['Statistics Indicator Description (0033)','Exception Description (0033)'], inplace=True)
HIRE['ExceptionTypeGroup'] = GTYPE['HireTypeGroup'].set_index(['Statistics Indicator Description (0033)','Exception Description (0033)']).ExceptionTypeGroup
HIRE.reset_index(inplace=True)
HIRE.columns = HIRE.columns.str.strip()

In [61]:
HIRE.head()

,Statistics Indicator Description (0033),Exception Description (0033),Serial Number,Subtype (0033),Statistics indicator (0033),ExceptionTypeGroup
0,Hire type,Inexperienced Hire,72,9U02,3,Inexperienced Hire
1,Hire type,Inexperienced Hire,134,9U02,3,Inexperienced Hire
2,Source,Other Source,479,9U02,1,Other
3,Hire type,Experienced Hire,479,9U02,3,Experienced Hire
4,Source,Other Source,615,9U02,1,Other


In [71]:
HIRE= HIRE[['Serial Number','Statistics Indicator Description (0033)','ExceptionTypeGroup']]
HIRE.columns = ['Serial Number','StatIndicator','ExceptionTypeGroup']

KeyError: ignored

In [72]:
HIRE.head()

,Serial Number,StatIndicator,ExceptionTypeGroup
0,72,Hire type,Inexperienced Hire
1,134,Hire type,Inexperienced Hire
2,479,Source,Other
3,479,Hire type,Experienced Hire
4,615,Source,Other


#Pivot

In [75]:
HIRE.pivot_table(values='ExceptionTypeGroup', index=HIRE['Serial Number'], columns='StatIndicator', aggfunc='first').sort_index()

StatIndicator,Hire type,Source
Serial Number,,
72,Inexperienced Hire,None
134,Inexperienced Hire,None
479,Experienced Hire,Other
615,Experienced Hire,Other
694,Experienced Hire,Other
749,Experienced Hire,Referral
754,Experienced Hire,Other
794,Experienced Hire,Other
837,Inexperienced Hire,None
